In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
epochs = 30
batch_size = 64

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data',
                          train=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=batch_size, shuffle=True
)

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn. Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [17]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [18]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print(epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item())

In [22]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [23]:
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(epoch, test_loss, test_accuracy)

1 0 60000 0.0 0.581648588180542
1 12800 60000 21.321961620469082 0.5766047239303589
1 25600 60000 42.643923240938165 0.6386409401893616
1 38400 60000 63.96588486140725 0.49029821157455444
1 51200 60000 85.28784648187633 0.8008973598480225
1 0.5370295132637024 79.78
2 0 60000 0.0 0.670315682888031
2 12800 60000 21.321961620469082 0.6173853278160095
2 25600 60000 42.643923240938165 0.7514384388923645
2 38400 60000 63.96588486140725 0.4924357831478119
2 51200 60000 85.28784648187633 0.49498921632766724
2 0.48627057003974916 81.96
3 0 60000 0.0 0.6567038297653198
3 12800 60000 21.321961620469082 0.5328803062438965
3 25600 60000 42.643923240938165 0.49237242341041565
3 38400 60000 63.96588486140725 0.7028912305831909
3 51200 60000 85.28784648187633 0.350227028131485
3 0.4498420417308807 83.69
4 0 60000 0.0 0.5971434712409973
4 12800 60000 21.321961620469082 0.535759687423706
4 25600 60000 42.643923240938165 0.5092207193374634
4 38400 60000 63.96588486140725 0.5614836812019348
4 51200 60000 